<a href="https://colab.research.google.com/github/shreyanshrao/zopper/blob/main/zopper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the dataframe
df = pd.read_excel('Jumbo & Company_ Attach % .xls')

In [ ]:
print(df.head())
print(df.info())

      Branch               Store_Name   Dec   Nov   Oct   Sep   Aug
0  Delhi_Ncr      Delhi(Janakpuri) Br  0.23  0.17  0.16  0.25  0.24
1  Delhi_Ncr      Haryana(Gurgaon) Br  0.21  0.26  0.15  0.28  0.04
2  Delhi_Ncr     Up(Greater Noida) Br  0.25  0.36  0.30  0.41  0.43
3       Pune         Pune(Bhosari) Br  0.33  0.33  0.36  0.13  0.32
4    Gujarat  Ahmedabad(Maninagar) Br  0.19  0.11  0.14  0.21  0.17
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Branch      163 non-null    object 
 1   Store_Name  163 non-null    object 
 2   Dec         163 non-null    float64
 3   Nov         163 non-null    float64
 4   Oct         163 non-null    float64
 5   Sep         163 non-null    float64
 6   Aug         163 non-null    float64
dtypes: float64(5), object(2)
memory usage: 9.0+ KB
None


In [ ]:
import numpy as np

# Reload (just to be safe context is fresh)
df = pd.read_excel('Jumbo & Company_ Attach % .xls')

# 1. Unpivot/Melt
month_order = {'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
df_long = df.melt(id_vars=['Branch', 'Store_Name'], var_name='Month', value_name='Attach_Pct')
df_long['Month_Num'] = df_long['Month'].map(month_order)
df_long = df_long.sort_values(by=['Branch', 'Store_Name', 'Month_Num'])

In [ ]:
def calc_slope(series):
    if len(series) < 2:
        return 0
    x = np.arange(len(series))
    # Fit line: y = mx + c
    slope, _ = np.polyfit(x, series, 1)
    return slope

In [ ]:
def calc_variance(series):
    return np.var(series)

In [ ]:
store_metrics = df_long.groupby(['Branch', 'Store_Name']).agg(
    Avg_Attach_Pct=('Attach_Pct', 'mean'),
    Max_Attach_Pct=('Attach_Pct', 'max'),
    Min_Attach_Pct=('Attach_Pct', 'min'),
    Variance=('Attach_Pct', 'var'),
)

In [ ]:
def get_trend_slope(x):
    x = x.sort_values('Month_Num')
    return calc_slope(x['Attach_Pct'])

slopes = df_long.groupby(['Branch', 'Store_Name']).apply(get_trend_slope)
store_metrics['Trend_Slope'] = slopes

/tmp/ipython-input-3676004114.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slopes = df_long.groupby(['Branch', 'Store_Name']).apply(get_trend_slope)


In [ ]:
def classify_trend(slope):
    if slope > 0.01: return "Improving"
    if slope < -0.01: return "Declining"
    return "Stable"

store_metrics['Trend_Status'] = store_metrics['Trend_Slope'].apply(classify_trend)

In [ ]:
branch_metrics = df_long.groupby(['Branch']).agg(
    Branch_Avg_Attach=('Attach_Pct', 'mean')
)
branch_monthly = df_long.groupby(['Branch', 'Month']).agg(
    Branch_Monthly_Avg=('Attach_Pct', 'mean')
).reset_index()

In [ ]:
store_metrics = store_metrics.reset_index()
store_metrics = store_metrics.merge(branch_metrics, on='Branch', how='left')

In [ ]:
global_monthly_avg = df_long.groupby('Month')['Attach_Pct'].mean()
# Sort by month num for display
global_monthly_avg_sorted = global_monthly_avg.reindex(['Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# 6. Highlights


In [ ]:
best_branch = branch_metrics['Branch_Avg_Attach'].idxmax()
best_branch_val = branch_metrics['Branch_Avg_Attach'].max()

best_store_row = store_metrics.loc[store_metrics['Avg_Attach_Pct'].idxmax()]
worst_store_row = store_metrics.loc[store_metrics['Avg_Attach_Pct'].idxmin()]
inconsistent_store_row = store_metrics.loc[store_metrics['Variance'].idxmax()]



In [ ]:
# 7. Predict January (Moving Average of last 3 months: Oct, Nov, Dec)
# We need to filter for specific months
last_3_months = df_long[df_long['Month'].isin(['Oct', 'Nov', 'Dec'])]
jan_forecast = last_3_months.groupby(['Branch', 'Store_Name'])['Attach_Pct'].mean().reset_index()
jan_forecast.rename(columns={'Attach_Pct': 'Jan_Forecast'}, inplace=True)

store_metrics = store_metrics.merge(jan_forecast, on=['Branch', 'Store_Name'], how='left')

# Prepare outputs for display
print("Global Monthly Trends:\n", global_monthly_avg_sorted)
print("\nBranch Averages:\n", branch_metrics)
print("\nSample Store Metrics (First 5):\n", store_metrics.head())
print("\nBest Branch:", best_branch, best_branch_val)
print("Best Store:", best_store_row['Store_Name'], best_store_row['Avg_Attach_Pct'])
print("Most Inconsistent Store:", inconsistent_store_row['Store_Name'], inconsistent_store_row['Variance'])

# Save processed data for user
store_metrics.to_csv('Processed_Store_Analysis.csv', index=False)

Global Monthly Trends:
 Month
Aug    0.128589
Sep    0.167301
Oct    0.170920
Nov    0.217117
Dec    0.217239
Name: Attach_Pct, dtype: float64

Branch Averages:
            Branch_Avg_Attach
Branch                      
Delhi_Ncr           0.243682
Gujarat             0.134583
Mumbai              0.173474
Pune                0.276500
Telangana           0.118350
Thane               0.148600

Sample Store Metrics (First 5):
       Branch               Store_Name  Avg_Attach_Pct  Max_Attach_Pct  \
0  Delhi_Ncr       DELHI(ASHOK VIHAR)           0.064            0.17   
1  Delhi_Ncr     DELHI(KRISHNA NAGAR)           0.170            0.35   
2  Delhi_Ncr  DELHI(ROHINI Sector-16)           0.070            0.28   
3  Delhi_Ncr        Delhi(Budh Vihar)           0.390            0.45   
4  Delhi_Ncr            Delhi(Burari)           0.334            0.48   

   Min_Attach_Pct  Variance  Trend_Slope Trend_Status  Branch_Avg_Attach  \
0            0.00   0.00773        0.049    Improving    